In [1]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re # 정규식 표현을 위한 모듈


# 윈도우용 크롬 웹드라이버 실행 경로 (Windows) 지정
excutable_path = "chromedriver.exe"
driver = webdriver.Chrome(executable_path=excutable_path)

# 사이트의 html 구조에 기반하여 크롤링을 수행
source_url = "https://namu.wiki/RecentChanges" # 크롤링할 사이트 주소를 정의
driver.get(source_url)  # 크롬 드라이버를 통해 URL의 HTML 문서 가져옴

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "app")))


req = driver.page_source
soup = BeautifulSoup(req, "html.parser") # BeautifulSoup의 soup 객체로 변환

#contents_table = soup.find(name="table")  
#table_body = contents_table.find(name="tbody")
#table_rows = table_body.find_all(name="tr")
table_rows = soup.select("table tbody tr")

# [페이지 링크주소 리스트 가져오기]

In [3]:
page_url_base = "https://namu.wiki" # 베이스 URL 정의
page_urls = [] # href 속성값을 담기 위한 빈 리스트 생성 

for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all("td")[0]
    td_url = first_td.find_all("a")
    if len(td_url) > 0:
        # 특정 속성 선택시 attrs["속성명"] 또는 get("속성명") 사용
        # page_url = page_url_base + td_url[0].get("href")
        # attrs는 딕셔너리 형태로 속성명과 속성값을 불러옴
        # attrs["href"]는 attrs 결과 중 key가 href인 것의 값만 불러옴
        page_url = page_url_base + td_url[0].attrs["href"] 
        if "png" not in page_url:
            page_urls.append(page_url)
            print(page_urls)

['https://namu.wiki/w/%EA%B9%80%EC%9D%B4%EB%8B%B4']
['https://namu.wiki/w/%EA%B9%80%EC%9D%B4%EB%8B%B4', 'https://namu.wiki/w/%EB%82%98%EB%82%98%EC%8B%9C%20%EB%AC%B4%EB%A9%94%EC%9D%B4']
['https://namu.wiki/w/%EA%B9%80%EC%9D%B4%EB%8B%B4', 'https://namu.wiki/w/%EB%82%98%EB%82%98%EC%8B%9C%20%EB%AC%B4%EB%A9%94%EC%9D%B4', 'https://namu.wiki/w/%EA%B5%90%EB%A7%8C%EC%A7%80%EC%99%95']
['https://namu.wiki/w/%EA%B9%80%EC%9D%B4%EB%8B%B4', 'https://namu.wiki/w/%EB%82%98%EB%82%98%EC%8B%9C%20%EB%AC%B4%EB%A9%94%EC%9D%B4', 'https://namu.wiki/w/%EA%B5%90%EB%A7%8C%EC%A7%80%EC%99%95', 'https://namu.wiki/w/%EC%B9%B4%EB%A5%BC%EB%A1%9C%EC%8A%A4%20%EC%88%98%EB%B2%A0%EB%A1%9C/2022%EB%85%84']
['https://namu.wiki/w/%EA%B9%80%EC%9D%B4%EB%8B%B4', 'https://namu.wiki/w/%EB%82%98%EB%82%98%EC%8B%9C%20%EB%AC%B4%EB%A9%94%EC%9D%B4', 'https://namu.wiki/w/%EA%B5%90%EB%A7%8C%EC%A7%80%EC%99%95', 'https://namu.wiki/w/%EC%B9%B4%EB%A5%BC%EB%A1%9C%EC%8A%A4%20%EC%88%98%EB%B2%A0%EB%A1%9C/2022%EB%85%84', 'https://namu.wiki/w/%EC%A7%

In [4]:
td_url[0].attrs

{'data-v-349171da': '', 'href': '/w/%EA%B9%80%EC%84%B1%ED%9D%AC(1989)'}

[각 링크 페이지내 텍스트 구조를 확인하여 제목, 카테고리, 내용 출력\]

In [5]:
#윈도우용 크롬 웹드라이버 실행 경로 (windows)지정
excutable_path="chromedriver.exe"
driver=webdriver.Chrome(executable_path=excutable_path)
#크롬 드라이버를 통해 page_urls[0]번째 사이트의 HTML 문서 가져옴
driver.get(page_urls[1])
req=driver.page_source#페이지 소스 저장
soup=BeautifulSoup(req,'html.parser')#html parser로 파싱
contents_table = soup.find(name='article') # 불러온 소스에서 태그명이 article인 요소 하나만 추출

# 타이틀 추출
title = contents_table.find_all('h1')[0]

# 카테고리 추출
category = contents_table.find_all('ul')[0]

# find_all 사용시
# contents_table.find_all(name = ''div, attrs = {'class':''wiki-paragraph}) 
# select 사용시
cotent_paragraphs = contents_table.select('div.wiki-paragraph') # div 태그 중 class 속성값이 wiki-paragraph인 요소를 추출

# 내용으로 추출한 리스트를 하나의 문자열로 전처리
content_corpus_list = [] # 내용 중 텍스트만 담을 빈 리스트 생성

for paragraphs in cotent_paragraphs:  # content_paragraphs 리스트의 값을 순서대로 paragraphs에 대입
    content_corpus_list.append(paragraphs.text) # 가져온 결과 태그 중 텍스트만 추출하영 content_corpus_list에 추가
    
content_corpus = ' '.join(content_corpus_list) # 텍스트.join(리스트명) => 리스트의 요소를 '텍스트'로 구분하여 하나의 문자열로 만듬

print(title.text) # 제목 출력
print('\n')
print(category.text) # 카테고리 출력
print('\n')
print(content_corpus) # 내용 출력

# 크롤링에 사용한 브라우저를 종료
driver.close()

나나시 무메이 


버츄얼 유튜버hololive ENGLISH/소속 크리에이터2021년 데뷔


+ 소속 멤버  [ hololive JP 멤버 펼치기 · 접기 ]0기생토키노 소라로보코 씨사쿠라 미코호시마치 스이세이AZKi1기생요조라 멜아키 로젠탈아카이 하아토시라카미 후부키나츠이로 마츠리2기생미나토 아쿠아무라사키 시온나키리 아야메유즈키 초코오오조라 스바루hololive GAMERS시라카미 후부키오오카미 미오네코마타 오카유이누가미 코로네3기생 - Hololive Fantasy우사다 페코라우루하 루시아시라누이 후레아시로가네 노엘호쇼 마린4기생 - holoForce아마네 카나타키류 코코츠노마키 와타메토코야미 토와히메모리 루나5기생 - holoFive유키하나 라미모모스즈 네네시시로 보탄마노 알로에오마루 폴카6기생 - 비밀결사 holoX라플라스 다크니스타카네 루이하쿠이 코요리사카마타 클로에카자마 이로하 0기생      토키노 소라 로보코 씨 사쿠라 미코 호시마치 스이세이 AZKi  1기생      요조라 멜 아키 로젠탈 아카이 하아토 시라카미 후부키 나츠이로 마츠리  2기생      미나토 아쿠아 무라사키 시온 나키리 아야메 유즈키 초코 오오조라 스바루  hololive GAMERS      시라카미 후부키 오오카미 미오 네코마타 오카유 이누가미 코로네   3기생 - Hololive Fantasy      우사다 페코라 우루하 루시아 시라누이 후레아 시로가네 노엘 호쇼 마린  4기생 - holoForce      아마네 카나타 키류 코코 츠노마키 와타메 토코야미 토와 히메모리 루나  5기생 - holoFive      유키하나 라미 모모스즈 네네 시시로 보탄 마노 알로에 오마루 폴카  6기생 - 비밀결사 holoX      라플라스 다크니스 타카네 루이 하쿠이 코요리 사카마타 클로에 카자마 이로하   [ hololive ID 멤버 펼치기 · 접기 ]1기생아윤다 리스무나 호시노바아이라니이오피프틴2기생쿠레이지 올리아냐 멜핏사파볼리아 레이네 1기생      아윤다 리스 무나 

[각각 링크 페이지를 크롤링하여 제목, 카테고리, 내용 출력]

In [6]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비
columns = ["title", "category", "content_text"]
df = pd.DataFrame(columns=columns)

#for page_url in page_urls:
for i in range(10):
    # 윈도우용 크롬 웹드라이버 실행 경로 (Windows) 지정
    excutable_path = "chromedriver.exe"
    driver = webdriver.Chrome(executable_path=excutable_path)
    # 크롬 드라이버를 통해 page_urls[0]번째 사이트의 HTML 문서 가져옴
    #driver.get(page_url)  # page_urls[i],  page_url의 정보를 가져옴
    driver.get(page_urls[i])  # page_urls[i],  page_url의 정보를 가져옴
    req = driver.page_source # 페이지 소스를 req에 저장
    soup = BeautifulSoup(req, 'html.parser') # html.parser로 파싱
    contents_table = soup.find(name="article") #  불러온 소스에서 태그명이 article인 요소 하나만 추출

    ### 타이틀 추출
    title = contents_table.find_all('h1')[0] # 태그명이 h1인 모든 태그 추출, article h1
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
        
    ### 카테고리 추출
    # 카테고리 정보가 없는 경우를 확인합니다.
    if len(contents_table.find_all("ul")) > 0: # article ul 로 검색한 결과 여러 ul 결과가 나올 경우
        category = contents_table.find_all("ul")[0] # 제일 첫번째 article ul 을 category로 설정
    else:
        category = None
        
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""

    ### 내용 추출
    #contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})  
    #div  태그 중  class 속성값이  wiki-paragraph인 요소를 추출
    content_paragraphs = contents_table.select("div.wiki-paragraph")  
    #  내용으로 추출한 리스트를 하나의 문자열로 전처리
    content_corpus_list = [] # 내용 중 텍스트만 담을 빈 리스트 생성
    
    # content_paragraphs 리스트의 값을 순서대로 paragraphs에 대입
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n", " "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")

    # 모든 정보를 하나의 데이터 프레임에 저장하기 위해서 시리즈 생성
    # 각 페이지의 정보를 추출하여 제목, 카테고리, 내용 순으로 행을 생성
    row = [row_title, row_category, "".join(content_corpus_list)]
    # 시리즈로 만듦
    series = pd.Series(row, index=df.columns)
    # 데이터 프레임에 시리즈를 추가, 한 페이지 당 하나의 행 추가
    df = df.append(series, ignore_index=True)
    
    # 크롤링에 사용한 브라우저를 종료합니다.
    driver.close()

In [7]:
df

,title,category,content_text
0,김이담,1988년 출생한국 남배우티버드2014년 데뷔대한민국의 개신교 신자뮤지컬 남배우,영한나라태오석준동욱음반 • 활동김이담Kim Yi Dam본명김영한출생1988년 11월...
1,나나시 무메이,버츄얼 유튜버hololive ENGLISH/소속 크리에이터2021년 데뷔,+ 소속 멤버 [ hololive JP 멤버 펼치기 · 접기 ]0기생토키노 소라로보...
2,교만지왕,마법천자문/등장인물,"암흑계 4대 마왕교만지왕다른 이름교만[1], 보라돌이[2], 거짓말쟁이[3], 쓰레..."
3,카를로스 수베로/2022년,야구감독/커리어한화 이글스/2022년,로그인 후 편집 가능한 문서입니다. 상위 문서: 카를로스 수베로1. 개요2...
4,지방우정청,우정사업본부,상위 문서: 우정사업본부 대한민국의ㅤㅤ 지방우정청 [ 펼치기ㆍ접기 ]서울지방우...
5,서균(고려),고려의 불교 신자이천 서씨1060년 출생1132년 사망,1. 개요2. 생애3. 가족관계3.1. 조상3.2. 자녀徐鈞(1060 ~ 1132)...
6,마왕루야,대한민국의 여성 인터넷 방송인버츄얼 유튜버트위치 스트리머2020년 데뷔나무위키 인터...,이 문서는 450자 이하의 토막글입니다.틀 적용 시 분류:토막글의 하위 분류 중 알...
7,아이아스급 전함,은하영웅전설/함선,아이아스급 대형전함Aias class flagship · アイアース 級旗艦OVA[1...
8,beatmania IIDX 28 BISTROVER/단위인정,beatmania,상위 문서: beatmania IIDX 28 BISTROVER전작과 구성이 동...
9,코우세이,죠죠의 기묘한 모험일본 만화/목록,"特急列車 コウセイ코우세이장르다크 판타지, 능력자 배틀, 어드벤처작가아라키 히로히코..."
